In [2]:
import numpy as np
import pandas as pd

from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from pathlib import Path 
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

In [3]:

csv = Path("kmeans").with_name('Markers.CSV')
x = pd.read_csv(csv)

data = x
# data.drop(columns=["PublicID"], inplace=True)  #dropping non int columns
data.drop(columns=["Unnamed: 0"], inplace=True)
data = data.fillna(0) # Have to double check this
data.drop(columns=["clusters"], inplace=True)


In [4]:
import numpy as np
import pandas as pd
from kmodes.kmodes import KModes
from scipy.cluster.hierarchy import linkage, dendrogram

# K-Modes Clustering
k = 5  # Number of clusters
kmodes = KModes(n_clusters=k, init="Huang", n_init=20, verbose=1)
kmodes.fit(data)

# Print cluster labels for K-Modes clustering
print("K-Modes Clustering:")
cluster_labels = kmodes.labels_
cluster_labels_series = pd.Series(cluster_labels)
print(cluster_labels_series.value_counts())

# for i, label in enumerate(kmodes.labels_):
#     print(f"Patient {i+1}: Cluster {label}")




Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 1430, cost: 3319.0
Run 1, iteration: 2/100, moves: 183, cost: 3319.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 3122.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 124, cost: 3328.0
Run 3, iteration: 2/100, moves: 27, cost: 3328.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 309, cost: 3067.0
Run 4, iteration: 2/100, moves: 313, cost: 2972.0
Run 4, iteration: 3/100, moves: 23, cost: 2972.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 413, cost: 2685.0
Run 5, iteration: 2/100, moves: 185, cost: 2685.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 6, iteration: 1/100